In [ ]:
!pip install -U torchtext==0.8.0 -q

     |████████████████████████████████| 7.0MB 7.2MB/s 


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import spacy
import pandas as pd
# from utils import translate_sentence, bleu, save_checkpoint, load_checkpoint
from torchtext.datasets import Multi30k
from torch.utils.tensorboard import SummaryWriter
from torchtext.data import Field, BucketIterator, TabularDataset
from sklearn.model_selection import train_test_split
import re

In [ ]:
#read the train csv file and make a validation file from it, 
#no need to run it 

# def make_val():
#     df = pd.read_csv('/content/drive/MyDrive/fourth_sem/capstone/t5/data/webNLG2020_train.csv')
#     train, val = train_test_split(df, test_size=0.2)
#     # print(len(train),len(val))
#     train.to_csv('/content/drive/MyDrive/fourth_sem/capstone/t5/data/train.csv')
#     val.to_csv('/content/drive/MyDrive/fourth_sem/capstone/t5/data/val.csv')
    
# make_val()
#read train val and test to perform initail cleaning
# def clean_df(df):
#     # print(df.columns)
#     cols = ['prefix','Unnamed: 0','Unnamed: 0.1']
#     for col in cols:
#         if col in df.columns:
#             df = df.drop(col,axis=1)
#     # df.rename(columns = {'input_text':'german', 'target_text':'english'}, inplace = True) 
#     return df

# def add_bar(text):
#     return text.replace(' '," | ")

# def mix(train, test):
#     df = pd.concat([train, test]).sample(frac=1).reset_index(drop=True)
#     train, test = train_test_split(df, test_size=0.1)
#     train = train.sample(frac=1).reset_index(drop=True)
#     test = test.sample(frac=1).reset_index(drop=True)
#     # print(len(train),len(test))
#     return train, test

# def read_tvt():
#     path = '/content/drive/MyDrive/fourth_sem/capstone/t5/data/'
#     wine = clean_df(pd.read_csv(path+'wine_triplets_text_uni.csv'))
#     wine['input_text'] = wine['input_text'].apply(lambda x: add_bar(x))
#     # print(wine.head(),len(wine))
#     #reading area and geograpy extracted triplets and sentences
#     area_and_geo = clean_df(pd.read_csv(path + 'area_and_geo.csv'))
#     train = clean_df(pd.read_csv(path+'webNLG2020_train.csv'))
#     #concat wine and area and geo with train dataset
#     train = pd.concat([train, wine]).sample(frac=1).reset_index(drop=True)
#     train = pd.concat([train, area_and_geo]).sample(frac=1).reset_index(drop=True)

#     # val = clean_df(pd.read_csv(path+'val.csv'))
#     test = clean_df(pd.read_csv(path+'webNLG2020_dev.csv'))
#     train, test = mix(train,test) 
#     train.to_csv(path+'train_t.csv',index=False)
#     # val.to_csv(path+'val_t.csv',index=False)
#     test.to_csv(path+'test_t.csv',index=False)
#     print(train.head())
#     print('train : ',len(train),'test :', len(test))
    
# read_tvt()

                                          input_text                                        target_text
0  Alvah_Sabin | party | Whig_Party_(United_State...  The largest city in Vermont is Burlington and ...
1  Aarhus_Airport | location | Tirstrup && Tirstr...  The location of Aarhus Airport is Tirstrup whi...
2  Aaron_Bertram | associatedBand/associatedMusic...  Aaron Bertram, who started performing in 1998 ...
3  Abilene,_Texas | isPartOf | Texas && United_St...  Abilene is a part of Texas, in the United Stat...
4  Aaron_Turner | associatedBand/associatedMusica...  Aaron Turner played with the bands Twilight an...
train :  36079 test : 4009


In [ ]:
spacy_eng = spacy.load("en")
count = 0
def tokenize_eng(text):
    global count
    if count % 1000 == 0:
        print(count)
    count += 1
    return [tok.text for tok in spacy_eng.tokenizer(text)]


input_text = Field(tokenize=tokenize_eng, lower=True, init_token="<sos>", eos_token="<eos>")
target_text = Field(tokenize=tokenize_eng, lower=True, init_token="<sos>", eos_token="<eos>")

fields = {'input_text':('src',input_text), 'target_text':('trg',target_text)}
# train_data, test_data = TabularDataset.splits(
#                                             path = '/content/drive/MyDrive/fourth_sem/capstone/t5/data/',
#                                             train = 'train_t.csv',
#                                             test = 'test_t.csv',
#                                             format = 'csv',
#                                             fields=fields)
train_data, test_data = TabularDataset.splits(
                                            path = '../data/all_merged/',
                                            train = 'train.csv',
                                            test = 'test.csv',
                                            format = 'csv',
                                            fields=fields)
input_text.build_vocab(train_data, max_size=10000, min_freq=2)
target_text.build_vocab(train_data, max_size=10000, min_freq=2)

/usr/local/lib/python3.7/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)
/usr/local/lib/python3.7/dist-packages/torchtext/data/example.py:52: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warn

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000


In [ ]:
# #dump the train and test data
# import json
# def save_examples(dataset, savepath):
#     with open(savepath, 'w') as f:
#         # Save num. elements (not really need it)
#         f.write(json.dumps(total))  # Write examples length
#         f.write("\n")

#         # Save elements
#         for pair in dataset.examples:
#             data = [pair.src, pair.trg]
#             f.write(json.dumps(data))  # Write samples
#             f.write("\n")


# def load_examples(filename):
#     examples = []
#     with open(filename, 'r') as f:
#         # Read num. elements (not really need it)
#         total = json.loads(f.readline())

#         # Save elements
#         for i in range(total):
#             line = f.readline()
#             example = json.loads(line)
#             # example = data.Example().fromlist(example, fields)  # Create Example obj. (you can do it here or later)
#             examples.append(example)

#     end = time.time()
#     print(end - start)
#     return examples

# save_examples(train_data,'/content/drive/MyDrive/fourth_sem/capstone/t5/data/train')

In [ ]:
cd /content/drive/MyDrive/fourth_sem/capstone/transformers

/content/drive/MyDrive/fourth_sem/capstone/transformers


In [ ]:
import torch
import spacy
from torchtext.data.metrics import bleu_score
import sys


def translate_sentence(model, sentence, input_text, target_text, device, max_length=50):
    spacy_ger = spacy.load("en") #changed loader from de to en

    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    if type(sentence) == str:
        tokens = [token.text.lower() for token in spacy_ger(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, input_text.init_token)
    tokens.append(input_text.eos_token)

    # Go through each input_text token and convert to an index
    text_to_indices = [input_text.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    outputs = [target_text.vocab.stoi["<sos>"]]
    for i in range(max_length):
        trg_tensor = torch.LongTensor(outputs).unsqueeze(1).to(device)

        with torch.no_grad():
            output = model(sentence_tensor, trg_tensor)

        best_guess = output.argmax(2)[-1, :].item()
        outputs.append(best_guess)

        if best_guess == target_text.vocab.stoi["<eos>"]:
            break

    translated_sentence = [target_text.vocab.itos[idx] for idx in outputs]
    # remove start token
    return translated_sentence[1:]


def bleu(data, model, input_text, target_text, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, input_text, target_text, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)


def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

In [ ]:
class Transformer(nn.Module):
    def __init__(
        self,
        embedding_size,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        forward_expansion,
        dropout,
        max_len,
        device,
    ):
        super(Transformer, self).__init__()
        self.src_word_embedding = nn.Embedding(src_vocab_size, embedding_size)
        self.src_position_embedding = nn.Embedding(max_len, embedding_size)
        self.trg_word_embedding = nn.Embedding(trg_vocab_size, embedding_size)
        self.trg_position_embedding = nn.Embedding(max_len, embedding_size)

        self.device = device
        self.transformer = nn.Transformer(
            embedding_size,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            forward_expansion,
            dropout,
        )
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = src_pad_idx

    def make_src_mask(self, src):
        src_mask = src.transpose(0, 1) == self.src_pad_idx

        # (N, src_len)
        return src_mask.to(self.device)

    def forward(self, src, trg):
        src_seq_length, N = src.shape
        trg_seq_length, N = trg.shape

        src_positions = (
            torch.arange(0, src_seq_length)
            .unsqueeze(1)
            .expand(src_seq_length, N)
            .to(self.device)
        )

        trg_positions = (
            torch.arange(0, trg_seq_length)
            .unsqueeze(1)
            .expand(trg_seq_length, N)
            .to(self.device)
        )

        embed_src = self.dropout(
            (self.src_word_embedding(src) + self.src_position_embedding(src_positions))
        )
        embed_trg = self.dropout(
            (self.trg_word_embedding(trg) + self.trg_position_embedding(trg_positions))
        )

        src_padding_mask = self.make_src_mask(src)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(
            self.device
        )

        out = self.transformer(
            embed_src,
            embed_trg,
            src_key_padding_mask=src_padding_mask,
            tgt_mask=trg_mask,
        )
        out = self.fc_out(out)
        return out

In [ ]:
# We're ready to define everything we need for training our Seq2Seq model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

load_model = False
save_model = False

# Training hyperparameters
num_epochs = 11
learning_rate = 3e-4
batch_size = 8

# Model hyperparameters
src_vocab_size = len(input_text.vocab)
trg_vocab_size = len(target_text.vocab)
embedding_size = 400
num_heads = 8
num_encoder_layers = 3 
num_decoder_layers = 3
dropout = 0.10
max_len = 150
forward_expansion = 4
src_pad_idx = target_text.vocab.stoi["<pad>"]

# # Tensorboard to get nice loss plot
# writer = SummaryWriter("runs/loss_plot")
# step = 0

train_iterator, test_iterator = BucketIterator.splits(
    (train_data,test_data),
    batch_size=batch_size,
    sort_within_batch=True,
    sort_key=lambda x: len(x.src),
    device=device,)

/usr/local/lib/python3.7/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [ ]:
model = Transformer(
    embedding_size,
    src_vocab_size,
    trg_vocab_size,
    src_pad_idx,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    forward_expansion,
    dropout,
    max_len,
    device,
    ).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)        

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, factor=0.1, patience=10, verbose=True
)

pad_idx = target_text.vocab.stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_index=pad_idx)


In [ ]:
model_name = str(num_epochs) + str(batch_size) + str(embedding_size) + str(num_heads) + str(num_encoder_layers) + str(num_decoder_layers) + '.pth.tar'
# num_epochs = 11
# batch_size = 8
# embedding_size = 400
# num_heads = 8
# num_encoder_layers = 3 
# num_decoder_layers = 3
if load_model:
    # load_checkpoint(torch.load("my_checkpoint.pth.tar"), model, optimizer)
    load_checkpoint(torch.load(mode_name), model, optimizer)

sentence = "Alan_Bean | nationality | United_States && Alan_Bean | occupation | Test_pilot && Alan_Bean | almaMater | UT Austin, B.S. 1955 && Alan_Bean | birthPlace | Wheeler,_Texas && Alan_Bean | timeInSpace | 100305.0 && Alan_Bean | selection | 1963 && Alan_Bean | status | Retired"

for epoch in range(num_epochs):
    print(f"[Epoch {epoch} / {num_epochs}]")

    if save_model:
        checkpoint = {
            "state_dict": model.state_dict(),
            "optimizer": optimizer.state_dict(),
        }
        save_checkpoint(checkpoint,model_name)

    model.eval()
    translated_sentence = translate_sentence(
        model, sentence, input_text, target_text, device, max_length=100
    )

    print(f"Translated example sentence: \n {translated_sentence}")
    model.train()
    losses = []

    for batch_idx, batch in enumerate(train_iterator):
        # Get input and targets and get to cuda
        inp_data = batch.src.to(device)
        target = batch.trg.to(device)
        print(inp_data.shape,target.shape)

        # Forward prop
        output = model(inp_data, target[:-1,:])

        # Output is of shape (trg_len, batch_size, output_dim) but Cross Entropy Loss
        # doesn't take input in that form. For example if we have MNIST we want to have
        # output to be: (N, 10) and targets just (N). Here we can view it in a similar
        # way that we have output_words * batch_size that we want to send in into
        # our cost function, so we need to do some reshapin.
        # Let's also remove the start token while we're at it
        output = output.reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)

        optimizer.zero_grad()

        loss = criterion(output, target)
        losses.append(loss.item())

        # Back prop
        loss.backward()
        # Clip to avoid exploding gradient issues, makes sure grads are
        # within a healthy range
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        # Gradient descent step
        optimizer.step()

        # plot to tensorboard
        # writer.add_scalar("Training loss", loss, global_step=step)
        # step += 1
    print("epoch loss ",losses[-1])
    mean_loss = sum(losses) / len(losses)
    scheduler.step(mean_loss)




[Epoch 0 / 11]
=> Saving checkpoint
Translated example sentence: 
 ['1974', 'schiphol', 'only', 'only', 'javanese', 'helped', 'schiphol', 'schiphol', 'tze', 'tze', 'tze', '1411.0', '1411.0', '1411.0', '253260', 'alter', 'only', 'romanesque', '.', 'alter', 'marjorie', 'schiphol', 'schiphol', '.0068', 'only', 'systems', 'limerick', 'developers', '103', 'brandon', 'only', '3800.0', 'ct', 'scrapped', 'only', 'schiphol', 'schiphol', 'only', '16244700.0', '253260', 'santilli', 'only', 'only', '1411.0', 'tze', 'therefore', 'gasparis', 'acura', 'schiphol', 'african', 'schiphol', 'tze', 'therefore', 'african', 'tze', 'tze', 'tze', 'tze', 'electronic', 'drastically', 'only', 'virginia', '1411.0', 'only', 'representative', 'on', 'gasparis', 'tze', 'jamali', 'on', '253260', '253260', 'elizabeth', 'developers', 'electronic', 'gasparis', '1411.0', '253260', 'rovers', 'african', 'schiphol', 'african', 'schiphol', 'african', 'only', 'african', 'african', 'only', 'brandon', 'only', 'african', 'african'

/usr/local/lib/python3.7/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


epoch loss  2.0104990005493164
[Epoch 1 / 11]
=> Saving checkpoint
Translated example sentence: 
 ['american', 'alan', 'bean', ',', 'who', 'graduated', 'from', 'the', 'university', 'of', 'texas', 'in', '1955', ',', 'graduated', 'from', 'ut', 'austin', 'with', 'a', 'b.s.', 'in', '1955', '.', 'he', 'graduated', 'from', 'the', 'university', 'of', 'texas', 'in', '1955', 'with', 'a', 'bachelor', 'of', 'science', 'degree', 'in', '1955', '.', '<eos>']
epoch loss  0.7371901869773865
[Epoch 2 / 11]
=> Saving checkpoint
Translated example sentence: 
 ['alan', 'bean', 'was', 'a', 'test', 'pilot', 'who', 'was', 'born', 'in', 'wheeler', ',', 'texas', '.', 'he', 'graduated', 'from', 'ut', 'austin', 'in', '1955', 'with', 'a', 'b.s.', 'in', '1955', 'with', 'a', 'b.s.', 'in', '1963', 'with', 'a', 'b.s.', 'he', 'was', 'a', 'test', 'pilot', '.', '<eos>']
epoch loss  1.488113522529602
[Epoch 3 / 11]
=> Saving checkpoint
Translated example sentence: 
 ['alan', 'bean', 'was', 'born', 'in', 'wheeler', ',', '

In [ ]:
# load_checkpoint(torch.load("my_checkpoint.pth.tar"), model, optimizer)
sentence = 'WhitehallLaneCabernetFranc | hasBody | medium'
translated_sentence = translate_sentence(
        model, sentence, input_text, target_text, device, max_length=100)
print(translated_sentence)

['whitehall', 'lane', 'cabernet', 'franc', 'has', 'medium', 'body', '.', '<eos>']


In [ ]:
# running on entire test data takes a while
score = bleu(test_data[0:500], model, input_text, target_text, device)
print(f"Bleu score {score * 100:.2f}")

Bleu score 33.87


In [ ]:
score = bleu(test_data[500:1000], model, input_text, target_text, device)
print(f"Bleu score {score * 100:.2f}")

Bleu score 33.50


In [ ]:
score = bleu(test_data[1000:1500], model, input_text, target_text, device)
print(f"Bleu score {score * 100:.2f}")

Bleu score 34.28


In [ ]:
score = bleu(test_data[1500:2000], model, input_text, target_text, device)
print(f"Bleu score {score * 100:.2f}")

Bleu score 32.34


In [ ]:
score = bleu(test_data[2000:2500], model, input_text, target_text, device)
print(f"Bleu score {score * 100:.2f}")

Bleu score 34.06


In [ ]:
score = bleu(test_data[2500:3000], model, input_text, target_text, device)
print(f"Bleu score {score * 100:.2f}")

Bleu score 32.54


In [ ]:
score = bleu(test_data[3000:3500], model, input_text, target_text, device)
print(f"Bleu score {score * 100:.2f}")

IndexError: ignored

In [ ]:
score = bleu(test_data[3500:4000], model, input_text, target_text, device)
print(f"Bleu score {score * 100:.2f}")

In [ ]:
# for sent in test_data[500:1000]:
#     print(sent.src,sent.trg)
#     translated_sentence = translate_sentence(model, sent.src, german, english, device, max_length=100)
#     print(translated_sentence)
#     break